In [936]:
import pandas as pd
import numpy as np
import warnings

warnings.simplefilter(action='ignore')

In [937]:
df = pd.read_csv('data.csv')

In [938]:
df.head()

,location,beds,price,area
0,3 Bedroom Independent House in J.P Nagar,3 BHK\n3 Baths,Price on Request,"1,800 sq.ft.\n(167 sq.m.) Built-up Area"
1,3 Bedroom Independent House in Dattagalli,3 BHK,"₹ 81 - 91 Lac\n₹ 4,525/sq.ft.","1,800-2,000 sq.ft.\n(167-186 sq.m.) Built-up Area"
2,3 BHK Residential Apartment in Ittige Gudu,3 BHK\n3 Baths,₹ 1.17 Cr,"1,260 sq.ft.\n(117 sq.m.) Carpet Area"
3,2 BHK Residential Apartment in Ittige Gudu,2 BHK\n2 Baths,₹ 78 Lac,840 sq.ft.\n(78 sq.m.) Carpet Area
4,4 BHK Residential Apartment in Bogadi,4 BHK,"₹ 62 Lac\n₹ 4,025/sq.ft.","1,540 sq.ft.\n(143 sq.m.) Super built-up Area"


In [939]:
df.isna().sum()

location    22
beds        26
price       22
area        22
dtype: int64

In [940]:
df.dropna(inplace=True)
df = df[~(df['price'] == 'Price on Request')]

In [941]:
df1 = pd.DataFrame()

In [942]:
df1['Type'] = ''
df1['Type'] = df['location'].apply(lambda x: 'House' if('house' in x.lower()) else x)
df1['Type'] = df1['Type'].apply(lambda x: 'Apartment' if('apartment' in x.lower()) else x)
df1['Type'] = df1['Type'].apply(lambda x: 'Villa' if('villa' in x.lower()) else x)
df1['Type'] = df1['Type'].apply(lambda x: 'Apartment' if('independent' in x.lower()) else x)

In [943]:
df1['Type'].unique()

array(['House', 'Apartment'], dtype=object)

In [944]:
df1['Location'] = [x.split('in')[1].strip() for x in df['location']]

In [945]:
df1['Beds'] = [int(x.split('B')[0].strip()) for x in df['beds']]

In [946]:
df1.rename(columns={'price':'Price', 'area':'Area','beds':'Beds','location':'Location'},inplace=True)

for i,x in enumerate(df['price']):
    #print(type(x))
    if x != 'Price on Request':
        if 'Cr' in x:
            df1['Price'][i] = x.split(' Cr')[0]
        elif 'Lac' in x:
             df1['Price'][i] = x.split(' Lac')[0]
        else:
            df1['Price'][i] = x
    else:
            df1['Price'][i] = x

KeyError: 'Price'

In [ ]:
df1 = df1[~(df1['Price'] == '₹ 25,000\n₹ 18/sq.ft.')]

In [ ]:
df1 = df1[~(df1['Price'] == '₹ 30,000\n₹ 1/sq.ft.')]

In [ ]:
df1['Price'] = df1['Price'].apply(lambda x : x.strip(' Lac\n') if('Lac\n' in x) else x)
df1['Price'] = df1['Price'].apply(lambda x : x.strip(' Cr'))

In [ ]:
df1['Price'].unique()

In [ ]:
df = df[~(df['price'] == '₹ 25,000\n₹ 18/sq.ft.')]
df = df[~(df['price'] == '₹ 30,000\n₹ 1/sq.ft.')]

In [ ]:
df1.Price.unique()

In [ ]:
for i,x in enumerate(df1['Price']):
    if '-' in x:
        print('1')
        df1['Price'][i] = float(x.split('-')[0]),float(x.split('-')[1])
    else:
        df1['Price'][i] = float(x)
        

        

In [ ]:
df1.Price

In [ ]:
for i,x in enumerate(df1['Price']):
    if type(x) == tuple:
        df1['Price'][i] = np.mean(x)
    elif type(x) == str:
        df1['Price'][i] = np.float(x)
    else:
        df1['Price'][i] = np.mean(x)

In [ ]:
df1['Price'] = df1['Price'].apply(lambda x : float(x) if(type(x) is str) else(x))

In [ ]:
df1['Price'] = df1['Price'].apply(lambda x : round(x*100,2) if(x < 10) else(x))

In [ ]:
df1['Price'].describe()

In [ ]:
df1['Area'] = [x.split('sq')[0] for x in df['area']]


In [ ]:
for i,x in enumerate(df1['Area']):
    #print(x)
    if '-' in x:
        temp1 = x.strip().split('-')[0].replace(',','')
        temp2 = x.strip().split('-')[1].replace(',','')
        temp1 = np.float(temp1)
        temp2 = np.float(temp2)
        avg = np.mean((temp1,temp2))
        df1['Area'][i] = avg
    else:
        temp = x.strip().replace(',','')
        df1['Area'][i] = np.float(temp)

In [ ]:
df1[df1['Area'] == '3,230 ']['Area'].index
df1.Area[528] = 3320

In [ ]:
df1[df1['Area'] == '600 ']['Area'].index
df1.Area[529] = 600

In [ ]:
df1.describe()

In [ ]:
type(df1['Area'][3])

In [ ]:
df1['Area'].describe()

In [ ]:
df1['Beds'].describe()

In [ ]:
df1[df1.Price < 200].hist()

In [ ]:
df1.shape

In [ ]:
df1.Area.unique()

In [ ]:
df1 = df1[~(df1.Area == 120000)]

In [ ]:
 df1[(df1.Area == 12000)]

In [ ]:
df1['PricePerSqft'] = df1['Price']/df1['Area']

In [ ]:
df1.drop('Price_per_sqft',1,inplace=True)

In [ ]:
df1.head()

In [ ]:
def remove_pps_ourliers(df):
    df_out = pd.DataFrame()
    for key, subdf in df.groupby('Location'):
        avg = np.mean(subdf['PricePerSqft'])
        stdev = np.std(subdf['PricePerSqft'])
        reduced_df = subdf[(subdf['PricePerSqft'] > (avg-stdev)) & (subdf.PricePerSqft < (avg+stdev)) ]
        df_out = pd.concat([df_out,reduced_df],ignore_index=True)
    return df_out

In [ ]:
df2 = remove_pps_ourliers(df1)

In [ ]:
df2.shape

In [ ]:
df2.Price.hist(bins=25)

In [ ]:
df2.describe()

In [ ]:
df1.describe()

In [ ]:
df2.to_csv('data_2_merge.csv',index=False)

In [ ]:
df1['Type'].size